# Imports

## System imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import json
import re

import seaborn as sns
import os

## Data imports

v1: predbezne vysledky po errate, ulozene na SharePointe OSN. Este sa budu updatovat

In [2]:
all_inpatients_file_path = r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\osn_vsetka_starostlivost_2023_ms_v2024.2.csv"

In [3]:
all_inpatients = pd.read_csv(all_inpatients_file_path, dtype=str, delimiter=";")

In [4]:
all_inpatients.head()

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms
0,35380_23000079,20,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-16,5130405,2002-04-25,522279,24,N61173014101,N6117301,N6117301,N61173,23000079,S05-34~S05-29
1,35380_23000214,19,NaN,NaN,NaN,j350,5g603&B&20230127,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-26,2023-01-31,2778465,2003-08-23,527696,24,N61173014101,N6117301,N6117301,N61173,23000214,S05-34~S05-29
2,35380_23000126,19,NaN,NaN,NaN,j350~j352,5g603&B&20230118~5g6071&Z&20230118,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-17,2023-01-21,3305786,2003-10-07,529249,24,N61173014101,N6117301,N6117301,N61173,23000126,S05-34~S05-29
3,35380_23000083,18,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-17,3305359,2004-08-15,520128,24,N61173014101,N6117301,N6117301,N61173,23000083,S53-33~S63-40
4,35380_23000159,18,NaN,NaN,NaN,h912,NaN,d61a,0.7048,DRG,h912,NaN,2023-01,2023-01,2023-01-20,2023-01-26,2852825,2004-08-31,520802,24,N61173014101,N6117301,N6117301,N61173,23000159,S53-29


In [4]:
all_inpatients.dtypes

id_hp                 object
vek_roky              object
vek_dni               object
hmotnost              object
upv                   object
diagnozy              object
vykony                object
drg                   object
erv                   object
typ_starostlivosti    object
diagnozy_z_01         object
vykony_z_01           object
obdobie_od            object
obdobie_do            object
datum_od              object
datum_do              object
id_poistenca          object
datum_narodenia       object
kod_pobytu            object
kod_zp                object
pzs_12                object
IDENTIFZAR            object
pzs_8                 object
pzs_6                 object
id_hp_pzs             object
ms                    object
dtype: object

In [5]:
all_inpatients.shape

(1178065, 26)

## Hospitals GPS

In [6]:
pzs_gps = pd.read_excel(r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\other\PZS_zakl_info_GPS_bezprepojeni.xlsx", dtype=str)

In [20]:
pzs_gps.head(2)

,IDENTIFZAR,PZS_short,NAZZAR,SIDZAR_KOD_OBEC,SIDZAR_KOD_OKRES_PZS,lat_pzs_ifexists,lng_pzs_ifexists,NAZZAR_povodne,lat_pzs_ifexists_povodne,lng_pzs_ifexists.1,SIDZAR_KOD_OBEC_povodne,SIDZAR_KOD_OKRES_PZS_povodne
0,P5848001,P58480,"Ministerstvo vnútra Slovenskej republiky, všeo...",501760,201,47.8001114823,17.6645174625,"Ministerstvo vnútra Slovenskej republiky, všeo...",47.8001114823,17.6645174625,501760,201
1,P5848002,P58480,"Ministerstvo vnútra Slovenskej republiky, všeo...",520004,702,48.9336706585,21.9065813542,"Ministerstvo vnútra Slovenskej republiky, všeo...",48.9336706585,21.9065813542,520004,702


## Patient residencies GPS

In [7]:
pacient_gps = pd.read_excel(r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\other\obce_gps_simple.xlsx")

In [22]:
pacient_gps.head()

,RESIDENCY,Obec_pacient_nazov,id_okres_pacient,Okres_pacient_nazov,okresne_mesto_zuj_pacient,okresne_mesto_pacient_nazov,lat_okres_pacient,lng_okres_pacient,residency_count
0,501280,Nesvady,401,Komárno,501026,Komárno,47.765799,18.119551,1
1,504025,Sala,405,Šaľa,504025,Šaľa,48.156715,17.875971,1
2,504092,Trnovec nad Vahom,405,Šaľa,504025,Šaľa,48.156715,17.875971,1
3,555509,Zalesie,108,Senec,508217,Senec,48.219813,17.396061,1
4,504131,Velke Ulany,202,Galanta,503665,Galanta,48.189953,17.725027,1


## MS names and levels

Tento zoznam treba "oholit" na uplne jednoduchu tabulku bez duplicit, inak to robi problemy a duplikuje riadky. To znamena, ze treba dat prec aj zdielane sluzby. Upravoval som este samotny excel.

In [8]:
ms_names_levels = pd.read_excel(r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\other\02_Zoznam-medicinskych-sluzieb 1 adjusted.xlsx")

In [9]:
ms_names_levels.head()

,Číslo programu,Kód medicínskej služby,ms_code,Zdieľaná medicínska služba,ms_name,typ,level_dospely,level_do1r,level_1az6,level_7az15,level_16az18
0,1,S01-04,S01-04,NaN,Bezvedomie a zabezpečenie DC vyžadujúce úzko š...,V,IV,NaN,NaN,NaN,NaN
1,1,S01-03,S01-03,NaN,Podpora dýchania vyžadujúca úzko špecializovan...,V,IV,NaN,NaN,NaN,NaN
2,1,S01-02,S01-02,NaN,Podpora krvného obehu vyžadujúca úzko špeciali...,V,IV,NaN,NaN,NaN,NaN
3,1,S01-01,S01-01,NaN,Kritické poruchy vnútorného prostredia vyžaduj...,V,IV,NaN,NaN,NaN,NaN
4,1,S01-14,S01-14,NaN,Komplexná intenzívna starostlivosť po transpla...,D,III,NaN,NaN,NaN,NaN


# Data processing

## Choose first MS 

In [10]:
all_inpatients['ms_code'] = all_inpatients['ms'].str.slice(0,6)

In [11]:
all_inpatients.head()

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms,ms_code
0,35380_23000079,20,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-16,5130405,2002-04-25,522279,24,N61173014101,N6117301,N6117301,N61173,23000079,S05-34~S05-29,S05-34
1,35380_23000214,19,NaN,NaN,NaN,j350,5g603&B&20230127,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-26,2023-01-31,2778465,2003-08-23,527696,24,N61173014101,N6117301,N6117301,N61173,23000214,S05-34~S05-29,S05-34
2,35380_23000126,19,NaN,NaN,NaN,j350~j352,5g603&B&20230118~5g6071&Z&20230118,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-17,2023-01-21,3305786,2003-10-07,529249,24,N61173014101,N6117301,N6117301,N61173,23000126,S05-34~S05-29,S05-34
3,35380_23000083,18,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-17,3305359,2004-08-15,520128,24,N61173014101,N6117301,N6117301,N61173,23000083,S53-33~S63-40,S53-33
4,35380_23000159,18,NaN,NaN,NaN,h912,NaN,d61a,0.7048,DRG,h912,NaN,2023-01,2023-01,2023-01-20,2023-01-26,2852825,2004-08-31,520802,24,N61173014101,N6117301,N6117301,N61173,23000159,S53-29,S53-29



## Merge MS names

In [13]:
ms_names_levels_NamesOnly = ms_names_levels[['ms_code', 'ms_name']]

In [14]:
ms_names_levels_NamesOnly.head()

,ms_code,ms_name
0,S01-04,Bezvedomie a zabezpečenie DC vyžadujúce úzko š...
1,S01-03,Podpora dýchania vyžadujúca úzko špecializovan...
2,S01-02,Podpora krvného obehu vyžadujúca úzko špeciali...
3,S01-01,Kritické poruchy vnútorného prostredia vyžaduj...
4,S01-14,Komplexná intenzívna starostlivosť po transpla...


In [15]:
all_inpatients = pd.merge(all_inpatients, ms_names_levels_NamesOnly, on='ms_code', how='left')

In [47]:
all_inpatients.head()

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms,ms_code,ms_name,ms_level,typ
0,35380_23000079,20,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-16,5130405,2002-04-25,522279,24,N61173014101,N6117301,N6117301,N61173,23000079,S05-34~S05-29,S05-34,Tonzilektómia alebo adenoidektómia,I,V
1,35380_23000214,19,NaN,NaN,NaN,j350,5g603&B&20230127,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-26,2023-01-31,2778465,2003-08-23,527696,24,N61173014101,N6117301,N6117301,N61173,23000214,S05-34~S05-29,S05-34,Tonzilektómia alebo adenoidektómia,I,V
2,35380_23000126,19,NaN,NaN,NaN,j350~j352,5g603&B&20230118~5g6071&Z&20230118,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-17,2023-01-21,3305786,2003-10-07,529249,24,N61173014101,N6117301,N6117301,N61173,23000126,S05-34~S05-29,S05-34,Tonzilektómia alebo adenoidektómia,I,V
3,35380_23000083,18,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-17,3305359,2004-08-15,520128,24,N61173014101,N6117301,N6117301,N61173,23000083,S53-33~S63-40,S53-33,Tonzilektómia alebo adenoidektómia,I,V
4,35380_23000159,18,NaN,NaN,NaN,h912,NaN,d61a,0.7048,DRG,h912,NaN,2023-01,2023-01,2023-01-20,2023-01-26,2852825,2004-08-31,520802,24,N61173014101,N6117301,N6117301,N61173,23000159,S53-29,S53-29,"Poruchy rovnováhy, strata sluchu alebo tinnitus",III,D


## Based on age, assign ms level

In [18]:
ms_names_levels_AllLevels = ms_names_levels[['ms_code', 'level_dospely', 'level_do1r', 'level_1az6', 'level_7az15', 'level_16az18']]

In [19]:
ms_names_levels_AllLevels.head(20)

,ms_code,level_dospely,level_do1r,level_1az6,level_7az15,level_16az18
0,S01-04,IV,NaN,NaN,NaN,NaN
1,S01-03,IV,NaN,NaN,NaN,NaN
2,S01-02,IV,NaN,NaN,NaN,NaN
3,S01-01,IV,NaN,NaN,NaN,NaN
4,S01-14,III,NaN,NaN,NaN,NaN
5,S01-13,III,NaN,NaN,NaN,NaN
6,S01-12,III,NaN,NaN,NaN,NaN
7,S01-11,III,NaN,NaN,NaN,NaN
8,S01-10,III,NaN,NaN,NaN,NaN
9,S01-09,III,NaN,NaN,NaN,NaN


In [20]:
all_inpatients = pd.merge(all_inpatients, ms_names_levels_AllLevels, on='ms_code', how='left')

In [23]:
def determine_ms_level(row):
    if row['vek_roky'] <= 1:
        return row['level_do1r']
    elif row['vek_roky'] <= 6:
        return row['level_1az6']
    elif row['vek_roky'] <= 15:
        return row['level_7az15']
    elif row['vek_roky'] <= 18:
        return row['level_16az18']    
    else:
        return np.nan

In [25]:
nan_count_vek_roky = all_inpatients['vek_roky'].isna().sum()
print(nan_count_vek_roky)

2


kedze su len 2 NaN values, dam ich prec, inak by bolo treba napisat funkciu, aby sa NaNs dali napr. na minus 1

In [26]:
all_inpatients = all_inpatients.dropna(subset=['vek_roky'])

In [27]:
all_inpatients['vek_roky'] = all_inpatients['vek_roky'].astype(int)

In [28]:
all_inpatients['ms_level'] = all_inpatients.apply(determine_ms_level, axis=1)

In [30]:
all_inpatients['ms_level'] = all_inpatients['ms_level'].fillna(all_inpatients['level_dospely'])

In [36]:
nan_count_ms_level = all_inpatients['ms_level'].isna().sum()
print(nan_count_ms_level)

98


In [38]:
filtered_df = all_inpatients.loc[all_inpatients['id_hp'] == "24283_23005692"]

In [39]:
filtered_df

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms,ms_code,ms_name,level_dospely,level_do1r,level_1az6,level_7az15,level_16az18,ms_level
1620,24283_23005692,17,NaN,NaN,NaN,r55,321030&Z&20230221~34017&Z&20230223~8u001&Z&202...,f73z,0.6039,DRG,r55,NaN,2023-02,2023-02,2023-02-20,2023-02-23,8773524,2005-02-24,524603,24,N33067007101,N3306701,N3306701,N33067,23005692,S68-24~S64-20,S68-24,MR v oblasti brucha a panvy,NaN,III,III,NaN,NaN,NaN


zostava mi stale okolo 100 NaNs, mozno chyba vo vykazovani, lebo take pripady nemaju level vo svojej kategorii podla veku a ani pre dospeleho. 

In [41]:
#later delete 
all_inpatients = all_inpatients.drop(columns=['level_dospely', 'level_do1r', 'level_1az6', 'level_7az15', 'level_16az18'])

In [42]:
all_inpatients.shape

(1178063, 29)

## Adding type of MS

In [44]:
ms_names_levels_msTypes = ms_names_levels[['ms_code', 'typ']]

In [45]:
all_inpatients = pd.merge(all_inpatients, ms_names_levels_msTypes, on='ms_code', how='left')

In [46]:
all_inpatients.shape

(1178063, 30)